In [1]:
# --- SAGEMAKER SETUP ---
# Import libraries Sagemaker (Container)

import boto3 # AWS SDK for Python. Used for direct S3 actions, clients, etc. (use it to upload CSVs) 
import os # filesystem / env var utilities.
import sagemaker # high-level SageMaker Python SDK. Provides Session, Estimator, helper classes and deployment tools
from sagemaker import get_execution_role # Convenience that returns the IAM role ARN when you run inside a SageMaker managed environment (Studio, notebook instance)
from sagemaker.serializers import CSVSerializer # Used by predictors to serialize inputs when calling a deployed endpoint (convert to text/csv)
from sagemaker.inputs import TrainingInput # Tells SageMaker where your training data lives (in S3) and how it should be loaded to .fit()

# Get the SageMaker session and the execution role from the SageMaker domain
sess = sagemaker.Session() # Creates an object that holds config (default S3 bucket, region) and helpers for uploading, describing jobs, etc.
role = get_execution_role() # An IAM role ARN that SageMaker uses to access S3, CloudWatch, ECR, etc. The role must have the right permissions (S3 write/read, SageMaker actions)

bucket = 'script-mode-xgb-demo' # The name of a bucket that is already created in S3
prefix = 'xgb-class-nba' # The name of the folder that will be created in the S3 bucket

print(f"Session bucket: {bucket}")
print(f"Prefix: {prefix}")
print("✅ SageMaker session ready")

Session bucket: script-mode-xgb-demo
Prefix: xgb-class-nba
✅ SageMaker session ready


In [2]:
# --- Read Training Dataset ---
import pandas as pd

# load dataset
df = pd.read_csv('datasets/training_data.csv')
df_copy = df.copy()
df_copy.head()

,row_id,player_id,age,position,team,opponent,minutes_played,points,rebounds,assists,steals,blocks,turnovers,fg_pct,three_pct,ft_pct,plus_minus,efficiency,game_location,rest_days,target
0,1,1102,25,C,NOP,MIA,12.2,9,3,0,1,1,2,51.4,19.6,79.9,-3,12,Home,2,0
1,2,1435,25,C,IND,HOU,30.9,15,14,5,1,2,0,41.0,31.2,68.1,1,37,Away,1,1
2,3,1348,30,PF,NYK,NOP,18.9,11,4,2,1,0,2,47.4,28.0,78.2,-2,16,Away,2,0
3,4,1270,27,SG,SAS,WAS,20.0,12,4,2,0,0,3,49.3,45.8,68.1,7,15,Away,2,0
4,5,1106,29,C,PHI,ORL,27.8,16,10,4,1,0,2,34.5,22.5,76.0,7,29,Away,1,1


In [3]:
# --- Defining CSVs and target for Training ---

from sklearn.model_selection import train_test_split

X_full = df_copy.drop(columns='target')
y_full = df_copy['target']

# Test Split
# 80/20
X_temp, X_test, y_temp, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=12345)

# Validation Set (0.25 of 0.8 = 0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=12345)
# The test size is 25% of the training data (80%), which is 20% of the full dataset
# 60 training /20 validation / 20 test 

# --- Export to CSV files ---
train_df = pd.concat([X_train, y_train], axis=1)
valid_df = pd.concat([X_valid, y_valid], axis=1)
test_df  = pd.concat([X_test, y_test], axis=1)

train_df.to_csv("datasets/train.csv", index=False)
valid_df.to_csv("datasets/validation.csv", index=False)
test_df.to_csv("datasets/test.csv", index=False)

print("✅ Local CSVs created successfully:")
print(f" - train.csv ({train_df.shape})")
print(f" - validation.csv ({valid_df.shape})")
print(f" - test.csv ({test_df.shape})")

✅ Local CSVs created successfully:
 - train.csv ((5400, 21))
 - validation.csv ((1800, 21))
 - test.csv ((1800, 21))


In [4]:
# --- Upload training, test, and validation data to the S3 bucket ---
# SageMaker can only access data stored in S3

# Path used: s3://<bucket>/<prefix>/train/train.csv, etc.
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('datasets/train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('datasets/validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('datasets/test.csv')

# Define the S3 inputs for SageMaker
# content_type is the input parsing: how the training container reads the input files once it downloads them from S3
s3_input_train = TrainingInput(
    s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv'
)
s3_input_validation = TrainingInput(
    s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv'
)
s3_input_test = TrainingInput(
    s3_data='s3://{}/{}/test/'.format(bucket, prefix), content_type='csv'
)

print(f"✅ Train data uploaded to: {s3_input_train.config['DataSource']['S3DataSource']['S3Uri']}")
print(f"✅ Validation data uploaded to: {s3_input_validation.config['DataSource']['S3DataSource']['S3Uri']}")
print(f"✅ Test data uploaded to: {s3_input_test.config['DataSource']['S3DataSource']['S3Uri']}")

✅ Train data uploaded to: s3://script-mode-xgb-demo/xgb-class-nba/train
✅ Validation data uploaded to: s3://script-mode-xgb-demo/xgb-class-nba/validation/
✅ Test data uploaded to: s3://script-mode-xgb-demo/xgb-class-nba/test/


In [5]:
# --- Testing the preprocessing script ---

# Allow imports from the same directory
import sys
sys.path.append(".")

# Import your custom preprocessing script
from importlib import reload
import preprocessing
reload(preprocessing)
from preprocessing import preprocess_data
import pandas as pd

train_raw = pd.read_csv("datasets/train.csv")
valid_raw = pd.read_csv("datasets/validation.csv")
test_raw  = pd.read_csv("datasets/test.csv")

train_processed = preprocess_data(train_raw, is_training=True)
valid_processed = preprocess_data(valid_raw, is_training=True)
test_processed = preprocess_data(test_raw, is_training=True)

print("Train:", train_processed.shape)
print("Valid:", valid_processed.shape)
print("Test:", test_processed.shape)

valid_processed.head()

print("\nColumns:", train_processed.columns.tolist())

Train: (5400, 65)
Valid: (1800, 65)
Test: (1800, 65)

Columns: ['efficiency', 'points', 'plus_minus', 'turnovers', 'blocks_per_min', 'turnovers_per_min', 'team_LAL', 'opponent_GSW', 'team_CHA', 'steals_per_min', 'team_MIA', 'steals', 'points_per_min', 'opponent_IND', 'opponent_CHA', 'opponent_DAL', 'blocks', 'position_SF', 'team_OKC', 'opponent_DEN', 'team_DAL', 'assists_per_min', 'team_BKN', 'team_CHI', 'opponent_LAL', 'efficiency_per_min', 'team_HOU', 'opponent_CLE', 'opponent_UTA', 'opponent_LAC', 'opponent_OKC', 'scoring_consistency', 'team_NOP', 'game_location_Home', 'age', 'assists', 'rebounds_per_min', 'team_GSW', 'usage_rate', 'team_MIN', 'opponent_MIL', 'opponent_NOP', 'age_bin_prime', 'three_pct', 'pm_bin_neutral', 'position_PG', 'team_PHI', 'opponent_NYK', 'opponent_ORL', 'minutes_played', 'team_SAC', 'opponent_DET', 'rest_days', 'fg_pct', 'team_TOR', 'impact_metric', 'ft_pct', 'team_DET', 'position_PF', 'team_CLE', 'rebounds', 'opponent_SAC', 'opponent_SAS', 'team_LAC', 'ta

In [6]:
# --- TRAINING THE MODEL ---

# --- Create XGBoost Estimator ---
from sagemaker.estimator import Estimator

# How to run the training job: which container, script, instance, and dependencies
xgb_estimator = Estimator(
    image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1',  # XGBoost container
    entry_point='train.py',          # The custom script
    source_dir='.',                  # Uploads train.py, preprocessing.py, etc.
    role=role,                       # IAM role for SageMaker
    instance_count=1,                # One instance
    instance_type='ml.m5.xlarge',    # Compute type 4 vCPUs, 16GB RAM
    base_job_name='nba-xgb-classifier',  # Job name prefix
    dependencies=['requirements.txt'] # An specific version of Pandas
)

# --- Launch training job ---
print("🚀 Launching XGBoost classification training job...")

# 1. SageMaker spins up a training instance (like ml.m5.xlarge).
# 2. Downloads the container image (in this case, the XGBoost image from ECR).
# 3. Uploads your scripts (train.py, requirements.txt, etc.) to the container.
# 4. Downloads your data from the S3 URIs you passed in (s3_input_train, etc.).
# 5. Runs the script inside the container

# Execution of the Training job
xgb_estimator.fit({
    'train': s3_input_train,
    'validation': s3_input_validation,
    'test': s3_input_test
})

print("✅ XGBoost training job launched successfully!")

🚀 Launching XGBoost classification training job...


INFO:sagemaker:Creating training-job with name: nba-xgb-classifier-2025-12-04-14-09-06-261


2025-12-04 14:09:11 Starting - Starting the training job...
2025-12-04 14:09:24 Starting - Preparing the instances for training...
2025-12-04 14:10:11 Downloading - Downloading the training image......
2025-12-04 14:11:02 Training - Training image download completed. Training in progress../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-12-04 14:11:09.472 ip-10-2-84-99.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-12-04 14:11:09.495 ip-10-2-84-99.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-12-04:14:11:09:INFO] Imported framework sagemaker_xgboost_container.training
[2025-12-04:14:11:09:INFO] No GPUs detected (normal if no gpus installed)
[2025-12-04:14:11:09:INFO] Invoking user training script.
[2025-12-0

In [7]:
# --- Where the model is (S3 SageMaker bucket) ---

import sagemaker

sess = sagemaker.Session()
job_name = "nba-xgb-classifier-2025-12-04-14-09-06-261"  # last job name
desc = sess.describe_training_job(job_name)
print("✅ Training job status:", desc["TrainingJobStatus"])
print("📦 Model S3 Path:", desc["ModelArtifacts"]["S3ModelArtifacts"])

✅ Training job status: Completed
📦 Model S3 Path: s3://sagemaker-us-east-1-059535450142/nba-xgb-classifier-2025-12-04-14-09-06-261/output/model.tar.gz


In [8]:
# --- Which was the container where the model was trained ---

sagemaker_session = sagemaker.session.Session()
job_name = 'nba-xgb-classifier-2025-12-04-14-09-06-261' # last job name

training_job = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=job_name)
print("📦 The Model Container is:",training_job['AlgorithmSpecification']['TrainingImage'])

📦 The Model Container is: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [11]:
# --- DEPLOYMENT AND INFERENCE WITH AN ENDPOINT ---

import os # Allow the interaction with the operating system: creating folders, reading env variables
import boto3 # AWS SDK for Python. Used for direct S3 actions, clients, etc. (use it to upload CSVs) 
import pandas as pd
from preprocessing import preprocess_data # Using the preprocessing script for feature engineering 
import sagemaker # High-level SageMaker Python SDK. Provides Session, Estimator, helper classes and deployment tools
from sagemaker import Session # Session represents the connection to the SageMaker environment
from sagemaker.model import Model
from sagemaker import get_execution_role

# Load the new dataset
df = pd.read_csv("datasets/blind_test_data.csv")
df_processed = preprocess_data(df, is_training=False)

# Save to a local CSV file for upload
local_file = "datasets/inference_data.csv"
df_processed.to_csv(local_file, header=False, index=False)
#df_processed.to_csv(local_file, header=True, index=False)
print("✅ The dataset for prediction is ready:")
print(df_processed)

# -------------------------------------
# Upload the dataset to the specific S3 Bucket
bucket_inference = 'script-mode-xgb-demo'  # The existing S3 bucket
prefix_inference = 'xgb-class-nba'    # Folder in the bucket
s3_key = os.path.join(prefix_inference, 'xgb_data_input', 'inference_data.csv')

s3 = boto3.client('s3') # Creates an S3 client using the boto3 library (AWS SDK for Python)
s3.upload_file(local_file, bucket_inference, s3_key) # Uploads the local file (local_file = "inference_data.csv") to S3
print(f"✅ Dataset uploaded to S3 at: s3://{bucket_inference}/{s3_key}")
# ---------------------------------------

# Deploy the model to a new endpoint, which creates the endpoint configuration and the endpoint
xgb_predictor = xgb_estimator.deploy( 
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge' ) 

print(f"✅ Model deployed successfully. Endpoint name: {xgb_predictor.endpoint_name}")

# Create a low-level client for the SageMaker Runtime 
sagemaker_runtime = boto3.client('sagemaker-runtime') # Sends requests to deployed endpoints and get real time predictions (inference) back

# Run inference
print("⚡ Running inference...")
# Open the file file.txt for reading, call it f, and automatically close it when I’m done — even if something goes wrong
with open(local_file, "r") as f: # Opens the local CSV file (inference_data.csv) that contains the new data you want predictions for
    payload = f.read() # Reads all its contents into a string variable called payload. Prepares the CSV as a raw text body ready to be sent over the network.

# Call the endpoint to get predictions
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=xgb_predictor.endpoint_name,
    Body=payload, # The actual data to send for prediction (the CSV)
    ContentType='text/csv' # Tells SageMaker how to interpret the payload (CSV format)
)

# --- Print predictions ---
predictions = response['Body'].read().decode('utf-8')
print("✅ Predictions received (resume):")
print(predictions[:220])  # Preview of the 220 characters of the variable predictions

# Convert raw string into a list of probabilities (floats)
probabilities = [float(p) for p in predictions.strip().split()]

# Convert probabilities into final classes using 0.5 threshold
predicted_classes = [1 if p >= 0.5 else 0 for p in probabilities]

# Create a DataFrame with both
df_output = pd.DataFrame({
    "probability": probabilities,
    "prediction": predicted_classes
})

print("First predictions:")
print(df_output.head(10))

# --- Save predictions to CSV file ---
#output_name = "predictions_from_endpoint.csv"
#with open(output_name, "w") as f: # Write 
    #f.write(predictions)
#print(f"Predictions saved locally as: {output_name}")
output_name = "predictions_from_endpoint.csv"
df_output.to_csv(output_name, index=False)
print(f"📁 Predictions saved locally as: {output_name}")

# Save only binary predictions
binary_name = "predictions_binary_only.csv"
pd.DataFrame({"prediction": predicted_classes}).to_csv(binary_name, index=False)
print(f"📁 Binary predictions saved locally as: {binary_name}")


# -------------------------------------
# Upload the predictions file to S3
s3_output_key = os.path.join(prefix_inference, 'xgb_prediction_output', 'predictions_from_endpoint.csv')
s3.upload_file(output_name, bucket_inference, s3_output_key)
print(f"✅ Predictions uploaded to S3 at: s3://{bucket_inference}/{s3_output_key}")

s3_output_binary_key = os.path.join(prefix_inference, 'xgb_prediction_output', binary_name)
s3.upload_file(binary_name, bucket_inference, s3_output_binary_key)
print(f"✅ Binary predictions uploaded to S3 at: s3://{bucket_inference}/{s3_output_binary_key}")
# -------------------------------------

# --- Clean up the endpoint ---
#print("🧹 Cleaning up endpoint to avoid extra charges...")
#try:
    #xgb_predictor.delete_endpoint()
   # print(f"✅ Endpoint '{xgb_predictor.endpoint_name}' deleted successfully.")
#except Exception as e:
    #print(f"⚠️ Could not delete endpoint: {e}")

print("---All steps completed successfully!---")

✅ The dataset for prediction is ready:
     efficiency  points  plus_minus  ...  opponent_SAC  opponent_SAS  team_LAC
0            12       7           2  ...             0             0         0
1            18      11          18  ...             0             0         0
2            27      20           2  ...             0             0         0
3            14      10           3  ...             0             0         0
4            14      12          -4  ...             1             0         0
..          ...     ...         ...  ...           ...           ...       ...
995          31      17           6  ...             0             0         0
996           7       3          12  ...             0             0         0
997          19      12          10  ...             0             0         1
998          31      22          12  ...             0             0         0
999          19      12          -2  ...             0             0         0

[1000 rows x

INFO:sagemaker:Creating model with name: nba-xgb-classifier-2025-12-04-14-30-00-504


✅ Dataset uploaded to S3 at: s3://script-mode-xgb-demo/xgb-class-nba/xgb_data_input/inference_data.csv


INFO:sagemaker:Creating endpoint-config with name nba-xgb-classifier-2025-12-04-14-30-00-504
INFO:sagemaker:Creating endpoint with name nba-xgb-classifier-2025-12-04-14-30-00-504


-------!✅ Model deployed successfully. Endpoint name: nba-xgb-classifier-2025-12-04-14-30-00-504
⚡ Running inference...
✅ Predictions received (resume):
0.008328422904014587
0.5703257918357849
0.6856430172920227
0.004911444615572691
0.008680814877152443
0.746567964553833
0.5245063900947571
0.9092857837677002
0.935229480266571
0.942295253276825
0.027252942323684692
0.2024
First predictions:
   probability  prediction
0     0.008328           0
1     0.570326           1
2     0.685643           1
3     0.004911           0
4     0.008681           0
5     0.746568           1
6     0.524506           1
7     0.909286           1
8     0.935229           1
9     0.942295           1
📁 Predictions saved locally as: predictions_from_endpoint.csv
📁 Binary predictions saved locally as: predictions_binary_only.csv
✅ Predictions uploaded to S3 at: s3://script-mode-xgb-demo/xgb-class-nba/xgb_prediction_output/predictions_from_endpoint.csv
✅ Binary predictions uploaded to S3 at: s3://script-mode